<a href="https://colab.research.google.com/github/Carloszone/Crypto_Token_Research/blob/main/Crypto_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
from sklearn import preprocessing
import os
import time
from google.colab import files
from datetime import date
from datetime import datetime
import statsmodels.api as sm
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Module

In [2]:
# module 1: LTE
# function to load data from df path
def load_df(token_path, transaction_path, index_path, commodity_path):
  token_df = pd.read_csv(token_path, parse_dates = ['Date'])
  transaction_df = pd.read_csv(transaction_path, parse_dates = ['Date'])
  index_df = pd.read_csv(index_path, parse_dates = ['Date'])
  commodity_df = pd.read_csv(commodity_path, parse_dates = ['Date'])
  return token_df, transaction_df, index_df, commodity_df


# function to convert unit from wei to ether
def wei_2_ether(num):
  res = num/10**18
  return res

# function to transfrom transaction_df
def transform_transaction_df(df):
  df['total_value_per_day'] =  df['total_value_per_day'].astype('float64')

  trans_df = df.copy()
  trans_df['total_value_per_day'] = df['total_value_per_day'].apply(lambda x: wei_2_ether(x))
  trans_df['avg_value_per_transaction'] = df['avg_value_per_transaction'].apply(lambda x: wei_2_ether(x)) #
  trans_df['total_gas_per_day'] = df['total_gas_per_day'].apply(lambda x: wei_2_ether(x)) #
  trans_df['total_gas_used_per_day'] = df['total_gas_used_per_day'].apply(lambda x: wei_2_ether(x)) #
  trans_df['avg_gas_used_per_transaction'] = df['avg_gas_used_per_transaction'].apply(lambda x: wei_2_ether(x)) #
  trans_df['total_transaction_fee'] = df['total_transaction_fee'].apply(lambda x: wei_2_ether(x))
  trans_df['total_priority_fee'] = df['total_priority_fee'].apply(lambda x: wei_2_ether(x))
  trans_df['avg_transaction_fee_per_transaction'] = df['avg_transaction_fee_per_transaction'].apply(lambda x: wei_2_ether(x)) #
  trans_df['avg_priority_fee_per_transaction'] = df['avg_priority_fee_per_transaction'].apply(lambda x: wei_2_ether(x)) #
  return trans_df

# function to fill na values with the previous non-na values
def fill_na(merged_df):
  df = merged_df.copy()
  # na check
  missing = df.isna().sum().sort_values(ascending = False)
  percent_missing = ((missing / df.isnull().count()) * 100).sort_values(ascending = False)
  missing_df = pd.concat([missing,percent_missing], axis = 1, keys = ['Total', 'Percent'],sort = False)

  # fill na
  columns = set(missing_df[missing_df['Total'] >= 1].reset_index()['index'])
  
  for col in columns:
    null_index = df.index[df[col].isnull() == True].tolist()
    for ind in null_index:
      df.loc[ind, col] = df.loc[ind-1, col]
  return df, missing_df

# function to merged dataframes
def merge_df(token_df, transaction_df, index_df, commodity_df):

  merged_df = token_df.merge(transaction_df, how = 'left', on = 'Date', suffixes = ('_token', '_transaction'))

  indexes = set(index_df['name'])
  commodities = set(commodity_df['name'])
  for index in indexes:
    merged_df = merged_df.merge(index_df[index_df['name'] == index], how = 'left', on = 'Date', suffixes = (None, f'_{index}'))
  for commodity in commodities:
    merged_df = merged_df.merge(commodity_df[commodity_df['name'] == commodity], how = 'left', on = 'Date', suffixes = (None, f'_{commodity}'))
   
  return merged_df

# function to extract needed data with regular expression
def extract_data(df, regex, fixed_list):
  columns = list(df.columns)
  target = re.compile(regex)
  target_list = list(filter(target.match, columns))

  if 'Date' in target_list or 'Date' in fixed_list:
    lists = target_list + fixed_list
  else:
    lists = ['Date'] + target_list + fixed_list
  res = df.loc[:,lists].set_index('Date')
  return res

# function to identify weekends
def weekend_check(_datetime):
  if date.weekday(_datetime) >=5:
    res = 1
  else:
    res = 0
  return res

# function to generate target df for the following process(visualizing and modeling)
def generate_target_df(token_path, transaction_path, index_path, commodity_path, fixed_list, regex, token_name):
  token_df, transaction_df, index_df, commodity_df = load_df(token_path, transaction_path, index_path, commodity_path)
  token_df = token_df[token_df.name == token_name]
  transaction_df = transform_transaction_df(transaction_df)
  merged_df = merge_df(token_df, transaction_df, index_df, commodity_df)
  merged_df, missing_df  = fill_na(merged_df)
  merged_df['weekend'] = merged_df['Date'].apply(weekend_check)
  fixed_list = fixed_list + list(transaction_df.columns)
  target_df = extract_data(merged_df, regex, fixed_list)
  return target_df, missing_df

# function to get df shift
def df_shift(dataset, y_label, shift_n = 1):
  df = dataset.copy()
  new_col = y_label + '_p'
  df[new_col] = df[y_label].shift(periods= shift_n)
  df = df.dropna()
  return df

# function to split df into trainset and testset
def split_df(df, timedate):
  trainset = df[df.index <= timedate]
  testset = df[df.index > timedate]
  return trainset, testset



In [3]:
# modelue 2: visualization

# funtion to draw lineplot
def draw_lineplot(df, cols, y, second_axis = True):
  plot_df = df.loc[:,cols]

  fig, ax1 = plt.subplots()
  if second_axis == True:
    ax2 = ax1.twinx()
    sns.lineplot(x = plot_df.index, y = y[0], ax = ax1, color = 'blue', label = y[0], data = plot_df)
    sns.lineplot(x = plot_df.index, y = y[1], ax = ax2, color = 'red', label = y[1], data = plot_df)
  else:
    ax1 = sns.lineplot(data = plot_df)
  ax1.tick_params(axis='x', rotation=90)
  ax1.legend(loc=0)
  ax2.legend(loc=0)
  plt.show()

# funtion to output correlation matrix
def correlation_matrix(df):
  plt.figure(figsize=(10, 10))
  corrMatrix = df.corr()
  ax = sns.heatmap(corrMatrix, annot=True)
  plt.show()


In [4]:
# module 3: linear regression model

# function to normalize df
def df_preprocessing(df, type = 'standardize'):
  X = df.values
  if type == 'standardize':
    std_scaler = preprocessing.StandardScaler().fit(X)
    x_scaled = std_scaler.transform(X)
    res = pd.DataFrame(x_scaled, columns=df.columns, index = df.index)
    return res, std_scaler
  elif type == 'minmax':
    minmax_scaler = preprocessing.MinMaxScaler().fit(X)
    x_scaled = minmax_scaler.transform(X)
    res = pd.DataFrame(x_scaled, columns=df.columns, index = df.index)
    return res, minmax_scaler
  

# function to get data for modelling
def get_data(df, y_label, previous = False, shift_n = 1, preprocess = None, intercept = True):
  if previous == True:
    df = df_shift(df, y_label = y_label, shift_n = shift_n)
  
  # 01 split X and Y
  X = df.loc[:, df.columns != y_label]
  Y = df.loc[:, df.columns == y_label]
  
  # 02 preprocess
  scaler = None
  if preprocess == 'standardize':
    X, scaler = df_preprocessing(X, type = 'standardize')
  if preprocess == 'minmax':
    X, scaler = df_preprocessing(X, type = 'minmax')

  # 03 add constant term
  if intercept == True:
    X = sm.add_constant(X)
  return X, Y, scaler

def alpha_search(x, y, alpha = None, type = 'simple'):
  if type == 'ridge':
    ridge_cv = RidgeCV(alphas= alpha)
    model_cv = ridge_cv.fit(x, y)
    return model_cv.alpha_
  if type == 'lasso':
    lasso_cv = LassoCV(alphas= alpha)
    model_cv = lasso_cv.fit(x, y)
    return model_cv.alpha_
 
def liner_model(X, Y, type = 'simple', alpha = None):
  model = sm.OLS(Y, X)
  results_fu = model.fit()
  Best_alpha = None
  if type == 'ridge':
    best_alpha = alpha_search(X, Y, alpha = alpha, type = 'ridge')
    model_ridge = model.fit_regularized(L1_wt=0, alpha= best_alpha, start_params=results_fu.params)
    ridge_result = sm.regression.linear_model.OLSResults(model, model_ridge.params, model.normalized_cov_params)
    return ridge_result, best_alpha
  elif type == 'lasso':
    best_alpha = alpha_search(X, Y, alpha = alpha, type = 'lasso')
    model_lasso = model.fit_regularized(L1_wt=1, alpha= best_alpha, start_params=results_fu.params)
    lasso_result = sm.regression.linear_model.OLSResults(model, model_lasso.params, model.normalized_cov_params)
    return lasso_result, best_alpha
  else:
    return results_fu, Best_alpha


@ignore_warnings(category=[ConvergenceWarning, UserWarning])
def backward_selection(df, y_label = 'Close', previous = False, shift_n = 1, preprocess = 'standardize', intercept = True, type = 'lasso', alpha = [0.01,0.05, 0.1, 0.5, 1], threshold = 0.05):
  X, Y, scaler = get_data(df, y_label = y_label, previous = previous, shift_n = shift_n, preprocess = preprocess, intercept = intercept)

  #create linear model
  model, best_alpha = liner_model(X, Y, type = 'simple', alpha = None)
  
  #backward selection model
  # .1 get feature coef result
  res = list(model.pvalues)
  max_p = max(res)

  # .2 find the biggest coef and correlated feature name
  while max_p > threshold:
    ind = res.index(max_p) # the index of max p value
    col = X.columns[ind] # find the column name

  # .3 remove the feature from X
    X = X.drop(col, axis = 1)
  # .4 build a new model
    if len(X.columns) == 0:
      print('all features have been removed, return the last avaiable model')
      return model, X, best_alpha, scaler
    model, best_alpha = liner_model(X, Y, type = 'simple', alpha = None)
    res = list(model.pvalues)
    max_p = max(res)

  #return result
  return model, X, best_alpha, scaler

In [5]:
# module 4: PCA model
def pca(x, feature_n = None):
  pca = PCA(n_components = feature_n)
  res = pca.fit_transform(x)
  return res

def pca_model(df, y_label, threshold = 0.05, feature_n = None):
  X, Y, scaler = get_data(df, y_label, preprocess = 'standardize', intercept = None)
  x = pca(X, feature_n)
  df_ = pd.concat([pd.DataFrame(x), pd.DataFrame({f'{y_label}':Y})], axis=1)
  x, y, model_result, remove_list, best_alpha = backward_selection(df_, y_label =  y_label, type = 'simple', alpha = None, threshold = threshold, preprocess = None, intercept = True)
  print(model_result.summary)
  return  x, y, model_result, scaler

In [6]:
# module 5: dataset and plot classes
class agg_df:
  def __init__(self, token_path, transaction_path, index_path, commodity_path, fixed_features, filter_features, token_name):
    self.token_path =  token_path
    self.transaction_path = transaction_path
    self.index_path = index_path
    self.commodity_path = commodity_path
    self.fixed_features = fixed_features
    self.filter_features = filter_features
    self.token_name = token_name

  def get_df(self):
    target_df, missing_df  = generate_target_df(self.token_path,
                                                self.transaction_path,
                                                self.index_path,
                                                self.commodity_path, 
                                                self.fixed_features, 
                                                self.filter_features, 
                                                self.token_name)
    return target_df

  def get_na_check(self):
    target_df, missing_df  = generate_target_df(self.token_path,
                                                self.transaction_path,
                                                self.index_path,
                                                self.commodity_path, 
                                                self.fixed_features, 
                                                self.filter_features, 
                                                self.token_name)
    print('NA check result:')
    print(missing_df[missing_df['Total'] >= 1])
    return missing_df


class plots:
  def __init__(self, df):
    self.df = df.copy()

  def get_lineplot(self, cols, y, second_axis = True):
    draw_lineplot(self.df , cols, y, second_axis = second_axis)

  def get_cor_matrix(self):
    correlation_matrix(self.df)

In [7]:
# module 6: model class
class backward_selection_model:
  def __init__(self, df, y_label = 'Close', previous = False, shift_n = 1, preprocess = 'standardize', intercept = True, type = 'lasso', alpha = [0.01,0.05, 0.1, 0.5, 1], threshold = 0.05):
    self.df = df.copy()
    self.y_label = y_label
    self.previous = previous
    self.shift_n = shift_n
    self.preprocess = preprocess
    self.intercept = intercept
    self.type = type
    self.alpha = alpha
    self.threshold = threshold
    model, X, best_alpha, scaler = backward_selection(self.df, y_label = self.y_label, previous = self.previous, 
                                                      shift_n = self.shift_n, preprocess = self.preprocess, intercept = self.intercept, 
                                                      type = self.type, alpha = self.alpha, threshold = self.threshold)
    self.model = model
    self.X = X
    self.best_alpha = best_alpha,
    self.scaler = scaler

  def get_model(self):
    return self.model
  
  def get_final_features(self):
    return self.X.columns
  
  def get_best_alpha(self):
    return self.best_alpha

  def get_scaler(self):
    return self.scaler

  def get_prediction(self, target_df):
    scaler = self.get_scaler()
    target_cols = list(self.get_final_features())
    model = self.get_model()

    if self.previous == True:
      target_df = df_shift(target_df, y_label = self.y_label, shift_n = self.shift_n)
  
    target_X = target_df.loc[:, target_df.columns != self.y_label]
    target_Y = target_df.loc[:, target_df.columns == self.y_label]
  
    if self.preprocess == 'standardize' or self.preprocess == 'minmax':
      X_ = scaler.transform(target_X)
      target_X = pd.DataFrame(X_, columns= target_X.columns, index =  target_X.index)

    if self.intercept == True:
      target_X = sm.add_constant(target_X)
    
    target_X = target_X.loc[:, target_cols]

    return model.predict(target_X)

  def get_mse(self, target_df):
    prediction = self.get_prediction(target_df)

    if self.previous == True:
      target_df = df_shift(target_df, y_label = self.y_label, shift_n = self.shift_n)

    target_y = target_df.loc[:, target_df.columns == self.y_label].values.ravel()

    return mse(target_y, prediction)

  def get_coef_df(self):
    return pd.DataFrame({'coef': self.model.params, 'P-value': self.model.pvalues})


In [8]:
token_path = 'https://raw.githubusercontent.com/Carloszone/Crypto_Token_Research/main/datasets/token_df.csv'
transaction_path = 'https://raw.githubusercontent.com/Carloszone/Crypto_Token_Research/main/datasets/transaction_df.csv'
index_path = 'https://raw.githubusercontent.com/Carloszone/Crypto_Token_Research/main/datasets/index_df.csv'
commodity_path = 'https://raw.githubusercontent.com/Carloszone/Crypto_Token_Research/main/datasets/commodity_df.csv'

fixed_features = ['Volume', 'weekend']
filter_features = re.compile('^Close.*')

tokens = ['Ethereum', 'Chainlink', 'Wrapped Bitcoin', '0x', 'Maker', 'Augur', 'Golem', 'Loopring', 'Basic Attention Token', 'Bitcoin', 'Binance Coin']

In [9]:
agg = pd.DataFrame({'Token': [], 'Model_Type': [], 'RMSE': [], 'R-Square Adj': []})

token = 'Ethereum'

token_info = agg_df(token_path, transaction_path, index_path, commodity_path, fixed_features, filter_features, token)
token_df = token_info.get_df()
# split df
train_df, test_df = split_df(token_df, timedate = '2021-11-06')

# build static model
print('Static Model')
s_model = backward_selection_model(train_df)
agg.loc[len(agg.index)] = [token, 'Static', np.sqrt(s_model.get_mse(test_df)), s_model.get_model().rsquared_adj]
filename = token + ' Static.csv'
res = s_model.get_coef_df()

Static Model


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [23]:
agg = pd.DataFrame({'Token': [], 'Model_Type': [],'Volume': [], 'Transaction Fee coef': [], 'Priority Fee coef': [], 'RMSE': [], 'R-Square Adj': []})

for token in tokens:
  print('\nToken name:', token)
  print('Process: ', tokens.index(token)+1,'/', len(tokens))

  # get token dataframe
  token_info = agg_df(token_path, transaction_path, index_path, commodity_path, fixed_features, filter_features, token)
  token_df = token_info.get_df()
  # split df
  train_df, test_df = split_df(token_df, timedate = '2021-11-06')

  # build static model
  s_model = backward_selection_model(train_df)
  res = s_model.get_coef_df()
  if 'Volume' in res.index:
    v_valume = res.loc['Volume',:][0]
  else:
    v_valume = 'NA'

  if 'avg_transaction_fee_per_transaction' in res.index:
    v_transaction = res.loc['avg_transaction_fee_per_transaction',:][0]
  else:
    v_transaction = 'NA'

  if  'avg_priority_fee_per_transaction' in res.index:
    v_priority = res.loc['avg_priority_fee_per_transaction',:][0]
  else:
    v_priority = 'NA'
  agg.loc[len(agg.index)] = [token, 'Static', v_valume, v_transaction, v_priority, np.sqrt(s_model.get_mse(test_df)), s_model.get_model().rsquared_adj]

  # build dynamic model
  d_model = backward_selection_model(train_df, previous = True)
  res = d_model.get_coef_df()
  if 'Volume' in res.index:
    v_valume = res.loc['Volume',:][0]
  else:
    v_valume = 'NA'

  if 'avg_transaction_fee_per_transaction' in res.index:
    v_transaction = res.loc['avg_transaction_fee_per_transaction',:][0]
  else:
    v_transaction = 'NA'

  if  'avg_priority_fee_per_transaction' in res.index:
    v_priority = res.loc['avg_priority_fee_per_transaction',:][0]
  else:
    v_priority = 'NA'
  agg.loc[len(agg.index)] = [token, 'Dynamic', v_valume, v_transaction, v_priority, np.sqrt(d_model.get_mse(test_df)), d_model.get_model().rsquared_adj]
agg.to_csv('result.csv')
files.download('result.csv')


Token name: Ethereum
Process:  1 / 11


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"



Token name: Chainlink
Process:  2 / 11


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"



Token name: Wrapped Bitcoin
Process:  3 / 11


...

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"



Token name: Binance Coin
Process:  11 / 11


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>